In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aacpassword"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encoding the image file
image_filename = 'Grazioso Salvare Logo.png'
with open(image_filename, 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
     # Display the image (logo)
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'})),

    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'All'}
        ],
        value='All',  # Default value is "All"
        style={'display': 'flex', 'flexDirection': 'column'}  # Forces vertical layout
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Adding user-friendly features
        page_size=10,  # Limit the number of rows displayed at a time
        sort_action="native",  # Enable sorting on the column headers
        filter_action="native",  # Allow filtering of data by each column
        row_selectable="single",  # Enable single row selection for interactions
        selected_rows=[0],  # Select the first row by default
        style_table={'height': '400px', 'overflowY': 'auto'},  # Set height and vertical scroll
        style_cell={'textAlign': 'left', 'padding': '5px'},  # Set text alignment and padding
        style_header={
            'backgroundColor': 'lightgrey',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},  # Alternate row color
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),
    html.Br(),
    html.Hr(),
    
    # Identifier 
    html.Div(
        children="Logan Riedell",
        style={
            'textAlign': 'center', 
            'marginTop': '20px', 
            'fontWeight': 'bold', 
            'fontSize': '18px',
            'color': 'darkblue'
        }
    ),
    
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################



    

## FIX ME Add code to filter interactive data table with MongoDB queries
#          
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)
@app.callback(Output('datatable-id', 'data'),
[Input('filter-type', 'value')])
def update_dashboard(filter_type):
    print(f"Filter type selected: {filter_type}")  # Debugging to check filter type

    # If 'All' is selected, display the full DataFrame
    if filter_type == 'All':
        filtered_df = df
    else:
        # Use the column name 'RescueType'
        filtered_df = df[df['RescueType'] == filter_type]
    
    return filtered_df.to_dict('records')
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    # Convert the viewData to a DataFrame
    dff = pd.DataFrame.from_records(viewData)
    
    # Ensure there is data to plot
    if not dff.empty:
        # Pie chart based on Age
        fig = px.pie(dff, names='Age', title='Animal Age Distribution')
        return dcc.Graph(figure=fig)
    else:
        return html.Div("No data available for the selected filter.")
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default to the first row if no row is selected
    if selected_rows is None or len(selected_rows) == 0:
        row = 0  # Default to row 0
    else:
        row = selected_rows[0]

    # Ensure latitude and longitude exist in the selected row
    latitude = dff.iloc[row].get('latitude', 30.75)  # Default value if latitude is not present
    longitude = dff.iloc[row].get('longitude', -97.48)  # Default value if longitude is not present
    breed = dff.iloc[row].get('Breed', 'Unknown')  
    animal_name = dff.iloc[row].get('Name', 'Unknown')  

    # Return the map component with updated marker position
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(breed),  # Display the breed as a tooltip
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)  # Display the animal's name in the popup
                ])
            ])
        ])
    ]


    
app.run_server(debug=True)


ModuleNotFoundError: No module named 'jupyter_dash'

###### 

In [13]:
import os
print(os.getcwd())
print(os.path.exists('Grazioso Salvare Logo.png'))

/home/loganriedell_snhu/Desktop
True


10 new animals with different locations inserted successfully.
